In [ ]:
import os
from sklearn.model_selection import train_test_split

# import torchvalues[0][1]
import cv2
import numpy as np
from torch.utils.data import Dataset
from torch.autograd import Variable
from PIL import Image
# from mypath import Path

class VideoDataset(Dataset):
    r"""A Dataset for a folder of videos. Expects the directory structure to be
    directory->[train/val/test]->[class labels]->[videos]. Initializes with a list
    of all file names, along with an array of labels, with label being automatically
    inferred from the respective folder names.
        Args:
            dataset (str): Name of dataset. Defaults to 'ucf101'.
            split (str): Determines which folder of the directory the dataset will read from. Defaults to 'train'.
            clip_len (int): Determines how many frames are there in each clip. Defaults to 16.
            preprocess (bool): Determines whether to preprocess dataset. Default is False.
    """

    def __init__(self, dataset='ucf101', split='train', clip_len=16, preprocess=False):
        self.root_dir, self.output_dir = Path.db_dir(dataset)
        print(self.root_dir)
        folder = os.path.join(self.output_dir, split)
        self.clip_len = clip_len
        self.split = split
        # The following three parameters are chosen as described in the paper section 4.1
        self.resize_height = 128
        self.resize_width = 171
        self.crop_size = 112

        if not self.check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You need to download it from official website.')

        if (not self.check_preprocess()) or preprocess:
            print('Preprocessing of {} dataset, this will take long, but it will be done only once.'.format(dataset))
            self.preprocess()

        # Obtain all the filenames of files inside all the class folders
        # Going through each class folder one at a time
        self.fnames, labels = [], []
        for label in sorted(os.listdir(folder)):
            for fname in os.listdir(os.path.join(folder, label)):
                self.fnames.append(os.path.join(folder, label, fname))
                labels.append(label)

        assert len(labels) == len(self.fnames)
        print('Number of {} videos: {:d}'.format(split, len(self.fnames)))

        # Prepare a mapping between the label names (strings) and indices (ints)
        self.label2index = {label: index for index, label in enumerate(sorted(set(labels)))}
        # Convert the list of label names into an array of label indices
        self.label_array = np.array([self.label2index[label] for label in labels], dtype=int)

        if dataset == "ucf101":
            if not os.path.exists('ucf_labels.txt'):
                with open('ucf_labels.txt', 'w') as f:
                    for id, label in enumerate(sorted(self.label2index)):
                        f.writelines(str(id+1) + ' ' + label + '\n')

        elif dataset == 'hmdb51':
            if not os.path.exists('hmdb_labels.txt'):
                with open('hmdb_labels.txt', 'w') as f:
                    for id, label in enumerate(sorted(self.label2index)):
                        f.writelines(str(id+1) + ' ' + label + '\n')


    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):
        # Loading and preprocessing.
        
        buffer = self.load_frames(self.fnames[index])

        buffer = self.crop(buffer, self.clip_len, self.crop_size)
        labels = np.array(self.label_array[index])
        
        if self.split == 'test':
            # Perform data augmentation
            buffer = self.randomflip(buffer)
        buffer = self.normalize(buffer)
        buffer = self.to_tensor(buffer)
        return torch.from_numpy(buffer), torch.from_numpy(labels)

    def check_integrity(self):
        if  os.path.exists(self.root_dir):
            return True
        else:
            return False

    def check_preprocess(self):
        # TODO: Check image size in output_dir
        if not os.path.exists(self.output_dir):
            return False
        elif not os.path.exists(os.path.join(self.output_dir, 'train')):
            return False

        for ii, video_class in enumerate(os.listdir(os.path.join(self.output_dir, 'train'))):
            for video in os.listdir(os.path.join(self.output_dir, 'train', video_class)):
                video_name = os.path.join(os.path.join(self.output_dir, 'train', video_class, video),
                                    sorted(os.listdir(os.path.join(self.output_dir, 'train', video_class, video)))[0])
                image = cv2.imread(video_name)
                if ii%5==0:
                  print("hello",type(image))
                if np.shape(image)[0] != 128 or np.shape(image)[1] != 171:
                    return False
                else:
                    break

            if ii == 10:
                break

        return True

    def preprocess(self):
        if not os.path.exists(self.output_dir):
            os.mkdir(self.output_dir)
            os.mkdir(os.path.join(self.output_dir, 'train'))
            os.mkdir(os.path.join(self.output_dir, 'val'))
            os.mkdir(os.path.join(self.output_dir, 'test'))

        # Split train/val/test sets
        for file in os.listdir(self.root_dir):
            file_path = os.path.join(self.root_dir, file)
            video_files = [name for name in os.listdir(file_path)]

            train_and_valid, test = train_test_split(video_files, test_size=0.2, random_state=42)
            train, val = train_test_split(train_and_valid, test_size=0.2, random_state=42)

            train_dir = os.path.join(self.output_dir, 'train', file)
            val_dir = os.path.join(self.output_dir, 'val', file)
            test_dir = os.path.join(self.output_dir, 'test', file)

            if not os.path.exists(train_dir):
                os.mkdir(train_dir)
            if not os.path.exists(val_dir):
                os.mkdir(val_dir)
            if not os.path.exists(test_dir):
                os.mkdir(test_dir)

            for video in train:
                self.process_video(video, file, train_dir)

            for video in val:
                self.process_video(video, file, val_dir)

            for video in test:
                self.process_video(video, file, test_dir)

        print('Preprocessing finished.')

    def process_video(self, video, action_name, save_dir):
        # Initialize a VideoCapture object to read video data into a numpy array
        video_filename = video.split('.')[0]
        if not os.path.exists(os.path.join(save_dir, video_filename)):
            os.mkdir(os.path.join(save_dir, video_filename))

        capture = cv2.VideoCapture(os.path.join(self.root_dir, action_name, video))

        frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Make sure splited video has at least 16 frames
        EXTRACT_FREQUENCY = 4
        if frame_count // EXTRACT_FREQUENCY <= 16:
            EXTRACT_FREQUENCY -= 1
            if frame_count // EXTRACT_FREQUENCY <= 16:
                EXTRACT_FREQUENCY -= 1
                if frame_count // EXTRACT_FREQUENCY <= 16:
                    EXTRACT_FREQUENCY -= 1

        count = 0
        i = 0
        retaining = True

        while (count < frame_count and retaining):
            retaining, frame = capture.read()
            if frame is None:
                continue

            if count % EXTRACT_FREQUENCY == 0:
                if (frame_height != self.resize_height) or (frame_width != self.resize_width):
                    frame = cv2.resize(frame, (self.resize_width, self.resize_height))
                cv2.imwrite(filename=os.path.join(save_dir, video_filename, '0000{}.jpg'.format(str(i))), img=frame)
                i += 1
            count += 1

        # Release the VideoCapture once it is no longer needed
        capture.release()

    def randomflip(self, buffer):
        """Horizontally flip the given image and ground truth randomly with a probability of 0.5."""

        if np.random.random() < 0.5:
            for i, frame in enumerate(buffer):
                frame = cv2.flip(buffer[i], flipCode=1)
                buffer[i] = cv2.flip(frame, flipCode=1)

        return buffer


    def normalize(self, buffer):
        for i, frame in enumerate(buffer):
            frame -= np.array([[[90.0, 98.0, 102.0]]])
            buffer[i] = frame

        return buffer

    def to_tensor(self, buffer):
        return buffer.transpose((3, 0, 1, 2))

    def load_frames(self, file_dir):
        frames = sorted([os.path.join(file_dir, img) for img in os.listdir(file_dir)])
        frame_count = len(frames)
        buffer = np.empty((frame_count, self.resize_height, self.resize_width, 3), np.dtype('float32'))
        for i, frame_name in enumerate(frames):
            frame = np.array(cv2.imread(frame_name)).astype(np.float64)
            buffer[i] = frame

        return buffer

    def crop(self, buffer, clip_len, crop_size):
        # randomly select time index for temporal jittering
        time_index = np.random.randint(buffer.shape[0] - clip_len)

        # Randomly select start indices in order to crop the video
        height_index = np.random.randint(buffer.shape[1] - crop_size)
        width_index = np.random.randint(buffer.shape[2] - crop_size)

        # Crop and jitter the video using indexing. The spatial crop is performed on
        # the entire array, so each frame is cropped in the same location. The temporal
        # jitter takes place via the selection of consecutive frames
        buffer = buffer[time_index:time_index + clip_len,
                 height_index:height_index + crop_size,
                 width_index:width_index + crop_size, :]

        return buffer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader
from torch.optim.lr_scheduler import StepLR
import torchvision
from torchvision import get_video_backend
from torchvision.models.video import r3d_18 
from torchvision import transforms
import os
from tqdm.auto import tqdm
import numpy as np
import time
# import av
import random
# print(f"PyAV version -- {av.__version__}")

In [ ]:
class Path(object):
    @staticmethod
    def db_dir(database):
        if database == 'ucf101':
            # folder that contains class labels
            root_dir = 'video_dataset'

            # Save preprocess data into output_dir
            output_dir = '/content/ucf101Preprocessed/'

            return root_dir, output_dir
        elif database == 'hmdb51':
            # folder that contains class labels
            root_dir = 'video_dataset'

            output_dir = '/content/hmdbPreprocessed'

            return root_dir, output_dir
        else:
            print('Database {} not available.'.format(database))
            raise NotImplementedError

    @staticmethod
    def model_dir():
        return 'Models/pretrainedModel1.pth'

In [ ]:
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar

--2022-06-01 05:35:38--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar [following]
--2022-06-01 05:35:39--  https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2124008126 (2.0G)
Saving to: ‘hmdb51_org.rar’

hmdb51_org.rar      100%[===================>]   1.98G  54.9MB/s    in 33s     

2022-06-01 05:36:11 (62.0 MB/s) - ‘hmdb51_org.rar’ saved [2124008126/2124008126]

--2022-06-01 05:36:12--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
R

In [ ]:
! mkdir -p video_dataset test_train_split
! unrar e test_train_splits.rar
! mv *.txt test_train_split
! rm test_train_splits.rar
! unrar e hmdb51_org.rar

! rm hmdb51_org.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from test_train_splits.rar

Extracting  brush_hair_test_split1.txt                                     0%  OK 
Extracting  brush_hair_test_split2.txt                                     1%  OK 
Extracting  brush_hair_test_split3.txt                                     1%  OK 
Extracting  cartwheel_test_split1.txt                                      2%  OK 
Extracting  cartwheel_test_split2.txt                                      2%  OK 
Extracting  cartwheel_test_split3.txt                                      3%  OK 
Extracting  catch_test_split1.txt                                          4%  OK 
Extracting  catch_test_split2.txt                                          4%  OK 
Extracting  catch_test_split3.txt                                          5%  OK 
Extracting  chew_test_split1.txt                                

In [ ]:
! mv *.rar video_dataset
import os
for files in os.listdir('video_dataset'):
    foldername = files.split('.')[0]
    os.system("mkdir -p video_dataset/" + foldername)
    os.system("unrar e video_dataset/"+ files + " video_dataset/"+foldername)

! rm video_dataset/*.rar

In [ ]:
dataset='hmdb51'

train_dataloader = DataLoader(VideoDataset(dataset=dataset, split='train',clip_len=16,preprocess=True), batch_size=20, shuffle=True, num_workers=2)
val_dataloader   = DataLoader(VideoDataset(dataset=dataset, split='val',  clip_len=16,preprocess=True), batch_size=20, num_workers=2)
test_dataloader  = DataLoader(VideoDataset(dataset=dataset, split='test', clip_len=16,preprocess=True), batch_size=20, num_workers=2)
#test3_dataloader= DataLoader(SampleDataset(dataset=dataset, split='test3',clip_len=16,transforms=test_transforms,preprocess=True), batch_size=20, shuffle=False, num_workers=2)
#train_dataloader2= DataLoader(SampleDataset(dataset=dataset, split='train',clip_len=16,transforms=train_transforms), batch_size=20, shuffle=False, num_workers=2)

video_dataset
Preprocessing of hmdb51 dataset, this will take long, but it will be done only once.
Preprocessing finished.
Number of train videos: 4289
video_dataset
hello <class 'numpy.ndarray'>
hello <class 'numpy.ndarray'>
hello <class 'numpy.ndarray'>
Preprocessing of hmdb51 dataset, this will take long, but it will be done only once.
Preprocessing finished.
Number of val videos: 1101
video_dataset
hello <class 'numpy.ndarray'>
hello <class 'numpy.ndarray'>
hello <class 'numpy.ndarray'>
Preprocessing of hmdb51 dataset, this will take long, but it will be done only once.
Preprocessing finished.
Number of test videos: 1376


In [ ]:
ModelR21D=torchvision.models.video.r2plus1d_18(pretrained=True) #pretrained=True was not working on 3rd april 2022
print(ModelR21D)

Downloading: "https://download.pytorch.org/models/r2plus1d_18-91a641e6.pth" to /root/.cache/torch/hub/checkpoints/r2plus1d_18-91a641e6.pth


  0%|          | 0.00/120M [00:00<?, ?B/s]

VideoResNet(
  (stem): R2Plus1dStem(
    (0): Conv3d(3, 45, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (1): BatchNorm3d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(45, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
    (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2Plus1D(
          (0): Conv3d(64, 144, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (1): BatchNorm3d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv3d(144, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
        )
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [ ]:
for param in ModelR21D.parameters(): #//commented on 12 April 2022
  param.requires_grad=False #//commented on 12 April 2022
ModelR21D.fc=nn.Linear(in_features=512,out_features=512)
old_fc=ModelR21D.fc
ModelR21D.fc=nn.Linear(old_fc.in_features,51)
print(ModelR21D)

VideoResNet(
  (stem): R2Plus1dStem(
    (0): Conv3d(3, 45, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (1): BatchNorm3d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(45, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
    (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2Plus1D(
          (0): Conv3d(64, 144, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (1): BatchNorm3d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv3d(144, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
        )
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(len(train_dataloader.dataset))
epochs=100000
print(device)

4289
cuda:0


In [ ]:
#this cell to be run if the model is starting to be trained
epoch_loss=[0]*epochs
epoch_acc=[0]*epochs
epochPsnr=[0]*epochs

In [ ]:
def returnOptimizer():
  lr=1e-3
  nn.init.xavier_uniform_(ModelR21D.fc.weight)
  params_to_update = []
  for name,param in ModelR21D.named_parameters():
      if param.requires_grad == True:
  # count=0
  # c=0
  # for child in ModelR21D.children():
  #   print('child',count,'is')
  #   print(child)
  #   count+=1
  #   if(count==5):
  #     for x in child:
  #       c+=1

  #       #if c==2:
  #       for param in x.parameters():
  #         #param.requires_grad=True
          params_to_update.append(param)
  optimizer = torch.optim.Adam(params_to_update,lr=lr)
  return optimizer

In [ ]:
#to be done after certain number of epochs,when the accuracy reaches certain level after fine tuning
count=0
c=0
for child in ModelR21D.children():
  print('child',count,'is')
  print(child)
  count+=1
  if(count==5):
      for x in child:
        c+=1

        if c==2:
          print('hello',child)

In [ ]:
for param in ModelR21D.parameters():
  print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True


In [ ]:

def train1(model,data,lossFunction1,optimizer1,epochs):
  model.to(device)
  
  
  for i in range(starting_epoch,epochs):
    
    running_loss=0
    running_acc=0
    running_psnr=0
    runningValacc=0
    runningValloss=0
    count=0;
    noise=0;
    loss2=0
    loss1=0
    model.train()
    images=[]
    
    for inputs, labels in tqdm(train_dataloader):
      count+=1
      #print("i =",i)
     
      print(inputs.size())
      inputs=Variable(inputs.float()).to(device)

      inputs=torch.permute(inputs,(0,2,1,3,4))
      print(inputs.size())
      labels=Variable(labels).to(device)
      loss=0
      optimizer1.zero_grad()
      outputs=model(inputs)
      prob=nn.Softmax(dim=1)(outputs)
      preds=torch.max(prob,1)[1]
      loss=lossFunction1(outputs,labels)
      
      #print(loss)
      loss.backward()
      optimizer1.step()
      
        #running_psnr+=computePSNR(noisy_img,inputs)
    
      running_loss += loss.item()*inputs.size(0)
      running_acc += torch.sum(preds==labels.data)   
          
    epoch_loss[i]=(running_loss)/len(train_dataloader.dataset)
    epoch_acc[i]=running_acc/len(train_dataloader.dataset)
    # epochPsnr[i]=running_psnr/(len(train_dataloader.dataset)/20)# this can be changed depending on the batch_size
    #scheduler.step()
    torch.save({"model_state_dict":model.state_dict(),
            "losses":epoch_loss,
            "accuracies":epoch_acc,
            "optimization_state_dict":optimizer1.state_dict(),
            "epochs":i},'/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/phase1Exp.pth')
    
    print(epoch_loss)
    print(epoch_acc)
    images.clear()
    count=0
   #validation
    # model.eval()
    # for inputs, labels in tqdm(val_dataloader):
    #   inputs=Variable(inputs).to(device)
    #   labels=Variable(outputs).to(device)
    #   with torch.no_grad():
    #     outputs=model(inputs)
    #     prob=nn.Softmax(dim=1)(outputs)
    #     preds=torch.max(prob,1)[1]
    #     loss=lossFunction(outputs,labels)
    #     runningValloss+=loss.item()*inputs.size(0)
    #     runningValacc+=torch.sum(pred==labels.data())
    # epochValacc=runningValacc/len(val_dataloader.dataset)
    # epochValloss=runningValloss/len(val_dataloader.dataset)
    # print(epochValacc,epochValloss)

    

In [ ]:

# lr=1e-3
# train_params = [{'params': get_1x_lr_params(model), 'lr': lr},
#                      {'params': get_10x_lr_params(model), 'lr': lr * 10}]
loss_fn1=torch.nn.CrossEntropyLoss()
# loss_fn2=meanSigmaLoss()
optimizer1=returnOptimizer()
# optimizer2=returnOptimizer()
scheduler=torch.optim.lr_scheduler.StepLR(step_size=20,optimizer=optimizer1,gamma=0.1)


In [ ]:
#checkpoint means loading saved model, initially starting epoch =0
checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/phase1Exp.pth')#,map_location=torch.device('cpu'))
ModelR21D.load_state_dict(checkpoint['model_state_dict'])
#optimizer1.load_state_dict(checkpoint['optimization_state_dict'])

epoch_loss = checkpoint['losses']
epoch_acc=checkpoint['accuracies']
starting_epoch=checkpoint['epochs']+1

# model.eval()

In [ ]:
print(epoch_loss)
print(epoch_acc)
print(starting_epoch)
# print(checkpoint['model_state_dict'])

In [ ]:
starting_epoch=0 #to be saved if training a previously trained model
train1(ModelR21D,train_dataloader,loss_fn1,optimizer1,epochs)

  0%|          | 0/215 [00:00<?, ?it/s]

torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20

  0%|          | 0/215 [00:00<?, ?it/s]

torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20

  0%|          | 0/215 [00:00<?, ?it/s]

torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20, 3, 16, 112, 112])
torch.Size([20

KeyboardInterrupt: ignored

In [ ]:
# for testing
ModelR21D.to(device)
ModelR21D.eval()
predictions=[]
true_labels=[]
testNoisyImages=[]
testImages=[]
c=0

testAcc=0
for inputs,labels in tqdm(test_dataloader):
  # noise=torch.zeros(inputs.size())
  # noise=noise.normal_(mean=mean,std=sigma)
  # noisy_img=noise+inputs
  inputs=Variable(inputs.float()).to(device)

  inputs=torch.permute(inputs,(0,2,1,3,4))
  labels=Variable(labels).to(device)
  # if(c<=3):
    
  #   testImages.append(inputs)

  
  
  # inputs=Variable(noisy_img.float()).to(device)
  # inputs=torch.permute(inputs,(0,2,1,3,4))
  # if(c<=3):
  #   testNoisyImages.append(inputs)

  true_labels.append(labels)
  c+=1
  preds=torch.max(nn.Softmax(dim=1)(ModelR21D(inputs)),1)[1]
  testAcc+=torch.sum(preds==labels)
  predictions.append(preds)

  0%|          | 0/69 [00:00<?, ?it/s]

In [ ]:
print(predictions)
print(true_labels)

[tensor([ 0,  0,  0, 43,  0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0, 49,
        39, 49], device='cuda:0'), tensor([18,  0, 14, 14,  1,  1,  1,  1, 16,  1, 14, 32, 14,  1, 14, 43, 14,  1,
        29, 14], device='cuda:0'), tensor([14, 14, 41, 14,  4,  8, 19,  8, 21, 43,  2, 13, 21,  2,  2, 16,  2, 34,
         1,  1], device='cuda:0'), tensor([ 2,  8, 21,  9,  2,  3, 39, 39,  3,  3, 10, 10, 48,  3, 39,  3,  3,  3,
         3,  3], device='cuda:0'), tensor([ 3,  8,  3, 10,  3,  3, 23, 16,  4,  4,  4,  4,  4, 49,  4, 33,  4,  4,
         4, 24], device='cuda:0'), tensor([49,  0,  4, 48, 12, 27, 23, 33,  8, 10,  4,  4,  0,  7,  5,  5,  5,  7,
         5,  5], device='cuda:0'), tensor([ 5,  5,  5,  7,  5,  5,  5,  5,  5,  5, 49,  5,  5,  5,  7,  6,  6,  6,
        49, 12], device='cuda:0'), tensor([ 6, 24,  6, 24, 32, 21,  6,  6, 32, 24,  6,  6, 26, 49, 24,  6, 49, 26,
         7, 19], device='cuda:0'), tensor([ 7, 19,  7, 23,  7,  7, 32, 19,  7, 21,  7, 32, 44,  7, 14,  7,  7,  7,

In [ ]:
print(testAcc/len(test_dataloader.dataset))

tensor(0.5000, device='cuda:0')


In [ ]:
print(train_dataloader.dataset[0][0].size())

torch.Size([3, 16, 112, 112])
